In [1]:
import numpy as np
import faiss

## Helper Functions

In [2]:
def semi_optimized_exhaustive_search(
        index_vectors: np.ndarray,
        query_vectors: np.ndarray,
        k: int,
):
    """
    This function performs an optimized exhaustive search.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        dim: The dimensionality of the vectors.
    Returns:
        An array of shape (n_queries, k) containing the indices of the k nearest neighbors for each query vector.
    """
    ann_lists = []
    for query_vec in query_vectors:
        distances = np.linalg.norm(index_vectors - query_vec, axis=1)
        ann_lists.append(list(np.argsort(distances)[:k]))
    return np.array(ann_lists)

In [3]:
def build_faiss_flatl2_index(
        index_vectors: np.ndarray,
        dim: int,
):
    """
    This function builds a Faiss flat L2 index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
    Returns:
        A Faiss flat L2 index.
    """
    index = faiss.IndexFlatL2(dim)
    index.add(index_vectors)
    return index

In [4]:
def faiss_search(
        query_vectors: np.ndarray,
        index: faiss.Index,
        k: int,
):
    """
    This function uses a Faiss index to search for the k-nearest neighbors of query_vectors.
    Args:
        query_vectors: An array of shape (n_queries, dim) containing the query vectors. 
        index: A Faiss index.
        k: The number of nearest neighbors to retrieve.
    Returns:
        An array of shape (, ) containing the indices of the k-nearest neighbors for each query vector.
    """
    distances, indices = index.search(query_vectors, k)
    return indices

In [5]:
def build_faiss_lsh_index(
        index_vectors: np.ndarray,
        dim: int,
        nbits: int,
):
    """
    This function builds a Faiss LSH index.
    Args:
        index_vectors: An array of shape (n_index, dim) containing the index vectors.
        dim: The dimensionality of the vectors. 
        nbits: The number of bits to use in the hash.
    Returns:
        A Faiss LSH index.
    """
    index = faiss.IndexLSH(dim, nbits)
    index.add(index_vectors)
    return index

In [6]:
def compute_recall_at_k(
        nn_gt: np.ndarray,
        ann: np.ndarray,
        k: int,
):
    """
    This function computes the recall@k.
    Args:
        nn_gt: The ground truth nearest neighbors.
        ann: The approximate nearest neighbors.
        k: The number of nearest neighbors to consider.
    Returns:
        The recall@k.
    """
    return round(sum([len(set(ann[i]) & set(nn_gt[i])) / k for i in range(len(ann))])/len(ann), 3)

# 2.1 -- LSH vs Naive Exhaustive Search (Regular Index Vectors)
### You just have to run the following cells:
* running time of the ground truth computation with semi_optimized_exhaustive_search (wall time)
* running time of creating faiss_lsh_index (wall time)
* running time of faiss_search over query_vectors with faiss_lsh_index (wall time)
* recall@10 for faiss_lsh_ann

In [7]:
query_vectors = np.load('data/query_vectors.npy')
index_vectors = np.load('data/index_vectors.npy')
k = 10
dim = index_vectors.shape[1]

In [8]:
%%time
gt_nn = semi_optimized_exhaustive_search(index_vectors, query_vectors, k)

CPU times: user 8.86 s, sys: 42 ms, total: 8.91 s
Wall time: 8.9 s


In [9]:
%%time
faiss_lsh_index = build_faiss_lsh_index(index_vectors, dim, nbits=2000)

CPU times: user 1.32 s, sys: 156 ms, total: 1.48 s
Wall time: 824 ms


In [10]:
%%time
faiss_lsh_ann = faiss_search(query_vectors, faiss_lsh_index, k)

CPU times: user 4.68 s, sys: 0 ns, total: 4.68 s
Wall time: 2.35 s


In [11]:
print(faiss_lsh_ann.shape)

(1000, 10)


In [12]:
print(f"recall@10 for faiss_lsh_index: {compute_recall_at_k(gt_nn, faiss_lsh_ann, k)}")

recall@10 for faiss_lsh_index: 0.11


# 2.2 -- Custom Indexing Algorithm
Build an indexing algorithm that satisfies the following requirements:
* The indexing algorithm should be able to handle vectors of different dimensions
* The running time of the indexing should be less than half of the running time of semi_optimized_exhaustive_search), reported in Section 2.1.
* The running time of searching over the index should be less than a third (1/3) of the time of the semi_optimized_exhaustive_search function, reported in Section 2.1.
* The performance (in terms of recall@10) of the indexing algorithm should be at least 0.25.

You are allowed to add as many helper functions as you need, as long as all of them appear in the next cell (the one containing custom_indexing_algorithm and custom_index_search). You cannot use faiss or scipy libraries for this task. Numpy is allowed. 

You can also test your algorithm with the additional two query-index sets by replacing the calls made few cells ago to:


In [13]:
#TODO: Write your code for 2.2.2 here
# You are allowed to add more arguments to the functions and create more functions if needed.
from collections import defaultdict

def initialize_random_centroids(K, X):
    """Initializes and returns k random centroids"""
    m, n = np.shape(X)
    centroids = np.empty((K, n))
    for i in range(K):
        # pick a random data point from X as the centroid
        centroids[i] =  X[np.random.choice(range(m))] 
    return centroids

def initialize_plus_centroids(K, X):
    """k-means++ initialisation – spreads centroids, boosts recall."""
    m, n = X.shape
    centroids = np.empty((K, n))
    centroids[0] = X[np.random.choice(m)]          # 1st centre – random
    dist2 = np.full(m, np.inf)                     # squared dists to nearest chosen
    for i in range(1, K):
        dist2 = np.minimum(dist2, np.sum((X - centroids[i-1])**2, axis=1))
        probs = dist2 / dist2.sum()                # D² sampling
        centroids[i] = X[np.random.choice(m, p=probs)]
    return centroids

def closest_centroid(x, centroids, K):
    """Finds and returns the index of the closest centroid for a given vector x"""
    distances = np.empty(K)
    for i in range(K):
        # compute distance from each centroid to a data point
        distances[i] = np.linalg.norm(centroids[i] - x)
    return np.argmin(distances) # return the index of the closest centroid

def assign_clusters(centroids, X):
    """Returns cluster indices for every sample."""
    dists = np.linalg.norm(X[:, None, :] - centroids[None, :, :], axis=2)
    return dists.argmin(axis=1)

def compute_means(cluster_idx, K, X):
    """Computes and returns the new centroids of the clusters"""
    _, n = np.shape(X)
    centroids = np.empty((K, n))
    for i in range(K):
        points = X[cluster_idx == i]
        if len(points) == 0:
            centroids[i] = X[np.random.choice(len(X))]  # reinit
        else:
            centroids[i] = np.mean(points, axis=0)
    return centroids
    

def run_k_means(K, X, max_iterations=100):
    """Runs the k-means algorithm and computes the final clusters"""
    centroids = initialize_plus_centroids(K, X)  # initialize random centroids
    for i in range(max_iterations):
        clusters = assign_clusters(centroids, X)  # assign data points to centroids
        previous_centroids = centroids.copy()                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
        centroids = compute_means(clusters, K, X)
        diff = previous_centroids - centroids
        if np.abs(diff).max() < 0.01:  # if there was no change - stop
            break
    return centroids, clusters


def custom_indexing_algorithm(index_vectors, dim, n_clusters):
    centroids, clusters = run_k_means(n_clusters, index_vectors)
    
    # build mapping from cluster_id to vectors
    inverted_lists = defaultdict(list)
    for i, cluster_id in enumerate(clusters.astype(int)):
        inverted_lists[cluster_id].append((index_vectors[i], i))  # store tuple: (vector, original index)
    
    for cluster_id in inverted_lists:
        vectors, indices = zip(*inverted_lists[cluster_id])
        inverted_lists[cluster_id] = (np.vstack(vectors), np.array(indices))
        
    return centroids, inverted_lists

def _gather_candidates(q, C, inv, L):
    d   = np.linalg.norm(C - q, axis=1)          # dist to each centroid
    cand = np.argsort(d)[:L]                # only top L centroids
    Vlst, Ilst = [], []
    for cid in cand:                             # build candidate pool
        if cid in inv:
            v, idx = inv[cid]
            Vlst.append(v)
            Ilst.append(idx)
    if not Vlst:                                 # empty handling
        return None, None
    return np.vstack(Vlst), np.concatenate(Ilst)


def custom_index_search(query_vectors, index, k, L=2):
    C, inv = index                       
    out = []
    for q in query_vectors:              # process each query 
        V, I = _gather_candidates(q, C, inv, L)  # pool of candidate vectors,ids
        if V is None:                    # no vectors found (empty clusters)
            out.append([])
            continue
        # rank all candidates by Euclidean distance to the query
        best = I[np.argsort(np.linalg.norm(V - q, axis=1))[:k]]
        out.append(list(best))           # store the top-k original indices
    return np.array(out)                 

In [14]:
# Add hyperparameters here (if needed)
N_CLUSTERS = 9

In [15]:
%%time
custom_index = custom_indexing_algorithm(index_vectors, dim, N_CLUSTERS)

CPU times: user 320 ms, sys: 948 ms, total: 1.27 s
Wall time: 1.23 s


In [16]:
%%time
custom_index_ann = custom_index_search(query_vectors, custom_index, k)

CPU times: user 2.39 s, sys: 8.03 ms, total: 2.39 s
Wall time: 2.39 s


In [17]:
print(f"recall@10 for custom_index_search: {compute_recall_at_k(gt_nn, custom_index_ann, k)}")

recall@10 for custom_index_search: 0.909
